In [3]:
import pandas as pd
import os

# Replace 'path_to_your_file.csv' with the actual path to your CSV file in Google Drive
current_dir = os.path.dirname(os.path.abspath(""))
file_path = os.path.join(current_dir, "data", "merged_data.csv")
data = pd.read_csv(file_path)

data['Difficulty For Model'] = data['Difficulty'] / (10 ** 13)

/Users/aidarpochanov/repos/zkml-solution/src


In [4]:
x = data[["Electricity_day_price", 'Difficulty For Model', 'HashRate', 'Power', 'Block Reward', 'Cost_2_months', 'Cost_3_months', 'Cost_4_months', 'Cost_5_months', 'Cost_6_months']].values

In [5]:
y1 = data[['BTC_needed_2_months']].values
y2 = data[['BTC_needed_3_months']].values
y3 = data[['BTC_needed_4_months']].values
y4 = data[['BTC_needed_5_months']].values

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y1_train, y1_test = train_test_split(x, y1, test_size=0.2, random_state=42)
X_train, X_test, y2_train, y2_test = train_test_split(x, y2, test_size=0.2, random_state=42)
X_train, X_test, y3_train, y3_test = train_test_split(x, y3, test_size=0.2, random_state=42)
X_train, X_test, y4_train, y4_test = train_test_split(x, y4, test_size=0.2, random_state=42)

In [7]:
# Increase the number of trees and maximum depth
n_estimators = 1  # Increase the number of trees
max_depth = 10  # Increase the maximum depth of each tree

xgb_reg_1 = xgb.XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
)
xgb_reg_1.fit(x, y1)

xgb_reg_2 = xgb.XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
)
xgb_reg_2.fit(x, y2)

xgb_reg_3 = xgb.XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
)
xgb_reg_3.fit(x, y3)

xgb_reg_4 = xgb.XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
)
xgb_reg_4.fit(x, y4)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
preds_1 = xgb_reg_1.predict(x)
preds_2 = xgb_reg_2.predict(x)
preds_3 = xgb_reg_3.predict(x)
preds_4 = xgb_reg_4.predict(x)


In [9]:
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.model_selection import RepeatedKFold

def run_evaluation(model, x, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    scores = absolute(scores)
    print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
    return scores

scores1 = run_evaluation(xgb_reg_1, x, y1)
scores2 = run_evaluation(xgb_reg_2, x, y2)
scores3 = run_evaluation(xgb_reg_3, x, y3)
scores4 = run_evaluation(xgb_reg_4, x, y4)


Mean MAE: 0.013 (0.000)
Mean MAE: 0.019 (0.000)
Mean MAE: 0.026 (0.000)
Mean MAE: 0.032 (0.000)


In [10]:
def compare(preds, y):
    print(f"preds={preds}\ny={y}")

compare(preds_1, y1)
compare(preds_2, y2)
compare(preds_3, y3)
compare(preds_4, y4)

preds=[0.0330119  0.03385348 0.03385348 ... 0.02162329 0.02162329 0.02162329]
y=[[0.04278764]
 [0.04398411]
 [0.04418531]
 ...
 [0.00460095]
 [0.0046036 ]
 [0.00482747]]
preds=[0.04840662 0.04840662 0.04971178 ... 0.03327667 0.03327667 0.03327667]
y=[[0.05943849]
 [0.06111033]
 [0.06158811]
 ...
 [0.00703943]
 [0.00708633]
 [0.00746538]]
preds=[0.06428805 0.06428805 0.06428805 ... 0.04543257 0.04543257 0.04543257]
y=[[0.07411282]
 [0.07582931]
 [0.07606297]
 ...
 [0.01048393]
 [0.01044179]
 [0.01094507]]
preds=[0.0784664  0.0784664  0.08071457 ... 0.05636442 0.05636442 0.05636442]
y=[[0.09188955]
 [0.09427039]
 [0.09460603]
 ...
 [0.01374353]
 [0.01371534]
 [0.01440978]]


In [13]:
from giza.zkcook import serialize_model
serialize_model(xgb_reg_1, "models/xgb_aidala_1.json")
serialize_model(xgb_reg_2, "models/xgb_aidala_2.json")
serialize_model(xgb_reg_3, "models/xgb_aidala_3.json")
serialize_model(xgb_reg_4, "models/xgb_aidala_4.json")